In [6]:
import ccxt
import pandas as pd
import numpy as np
import datetime

In [7]:
exchange = ccxt.binance()

def fetchOHLCV(symbol, timeframe, since, until=None):
	df = exchange.fetch_ohlcv(symbol=f'{symbol}', timeframe=timeframe, since=int(datetime.datetime.timestamp(since) * 1000), params={'until': until})
	df = pd.DataFrame(df, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
	df['datetime'] = pd.to_datetime(df['timestamp'] * 1000 * 1000, utc=True).dt.tz_convert('Australia/Adelaide')
	return df

five_days_ago = datetime.datetime.strptime('2025-02-07', '%Y-%m-%d')

df = fetchOHLCV('BTC/USDT', '1m', five_days_ago)

In [8]:
df = df.set_index('datetime')
df

,timestamp,Open,High,Low,Close,Volume
datetime,,,,,,
2025-02-07 02:30:00+10:30,1738857600000,97333.09,97347.71,97209.53,97293.01,34.43656
2025-02-07 02:31:00+10:30,1738857660000,97293.01,97410.30,97280.01,97410.29,26.71689
2025-02-07 02:32:00+10:30,1738857720000,97410.30,97434.79,97322.32,97425.33,27.01558
2025-02-07 02:33:00+10:30,1738857780000,97425.32,97545.79,97406.01,97537.26,28.13103
2025-02-07 02:34:00+10:30,1738857840000,97537.27,97537.66,97384.01,97487.70,32.81806
...,...,...,...,...,...,...
2025-02-07 10:45:00+10:30,1738887300000,96763.47,96805.97,96708.30,96805.97,11.83171
2025-02-07 10:46:00+10:30,1738887360000,96805.96,96836.00,96768.02,96794.10,10.57806
2025-02-07 10:47:00+10:30,1738887420000,96794.03,96794.03,96702.91,96767.37,7.52674


In [11]:
from hiearhical_extremes import HiearchicalExtremes

h = df['High'].to_numpy()
l = df['Low'].to_numpy()
c = df['Close'].to_numpy()

dc = HiearchicalExtremes(2, 60)
for i in range(len(h)):
    dc.update(i, df.index, h, l, c)

ext_df_1 = pd.DataFrame(dc.extremes[0])
ext_df_2 = pd.DataFrame(dc.extremes[1])

import plotly.graph_objects as go
# ... existing code ...
fig = go.Figure(data=[
    go.Scatter(x=ext_df_1['timestamp'], y=ext_df_1['price'], mode='lines', line=dict(color='blue')),
    go.Scatter(x=ext_df_2['timestamp'], y=ext_df_2['price'], mode='lines', line=dict(color='red')),
    go.Candlestick(x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close']),
])
# ... existing code ...
fig.show()

74 2025-02-07 03:44:00+10:30
98 2025-02-07 04:08:00+10:30
163 2025-02-07 05:13:00+10:30
235 2025-02-07 06:25:00+10:30
295 2025-02-07 07:25:00+10:30
346 2025-02-07 08:16:00+10:30
389 2025-02-07 08:59:00+10:30
462 2025-02-07 10:12:00+10:30
